<a href="https://colab.research.google.com/github/pengfei123xiao/ML_Basic/blob/master/models/Ch05-DecisionTree/RF_GBDT_XGBoost_LightGBM/LightGBM_Key_Points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LightGBM Key Points

## 1.LightGBM
    在2017年年1月微软在GitHub的上开源了一个新的升压工具--LightGBM。
    论文可参考，《LightGBM: A Highly Efficient Gradient BoostingDecision Tree》
    LightGBM它是对于XGB提升性能的版本，相对于其他GBM来说具有相近的准确率而且是其训练速度20倍。
    lightGBM实际上是GBDT采用上两种新方法：GOSS（Gradient-based One-Side Sampling, 基于梯度的单边采样）和 EFB（Exclusive Feature Bundling， 互斥特征捆绑）。
    相比XGBoost，LightGBM有以下优势。

## 2.LightGBM的起源
传统的boosting算法（如GBDT和XGBoost）已经有相当好的效率，但是在如今的大样本和高维度的环境下，传统的boosting似乎在效率和可扩展性上不能满足现在的需求了，主要的原因就是传统的boosting算法需要对每一个特征都要扫描所有的样本点来选择最好的切分点，这是非常的耗时。为了解决这种在大样本高纬度数据的环境下耗时的问题，出现了Lightgbm 。
Lightgbm使用了如下两种解决办法：一是GOSS（Gradient-based One-Side Sampling, 基于梯度的单边采样），不**使用所用的样本点来计算梯度，而是对样本进行采样来计算梯度**；二是EFB（Exclusive Feature Bundling， 互斥特征捆绑），这里不是使用所有的特征来进行扫描获得最佳的切分点，而是将**某些特征进行捆绑在一起来降低特征的维度**，使寻找最佳切分点的消耗减少。这样大大的降低的处理样本的时间复杂度，但在精度上，通过大量的实验证明，在某些数据集上使用Lightgbm并不损失精度，甚至有时还会提升精度。

## 3.Histogram VS pre-sorted
**3.1 优势**

Pre-sorted 算法需要的内存**约是训练数据的两倍**(2 * #data * #features* 4Bytes)，它需要用32位浮点(4Bytes)来保存 feature value，并且**对每一列特征，都需要一个额外的排好序的索引，这也需要32位(4Bytes)的存储空间**。因此是(2 * #data * #features* 4Bytes)。

而对于 histogram 算法，则只需要(#data * #features * 1Bytes)的内存消耗，仅为 pre-sorted算法的1/8。因为 histogram 算法仅需要存储 feature bin value (离散化后的数值)，不需要原始的 feature value，也不用排序，而 bin value 用 1Bytes(256 bins) 的大小一般也就足够了。

计算上的优势则是大幅**减少了计算分割点增益的次数**。对于每一个特征，pre-sorted 需要对每一个不同特征值都计算一次分割增益，代价是O(#feature*#distinct_values_of_the_feature)；而 histogram 只需要计算#bins次，代价是(#feature*#bins)。

还有一个很重要的点是cache-miss(slows down the overall process)。cache-miss对速度的影响是特别大的。预排序中有2个操作频繁的地方会造成cache miss，一是**对梯度的访问**，在计算gain的时候需要利用梯度，不同特征访问梯度的顺序都是不一样的，且是随机的，因此这部分会造成严重的cache-miss。二是对于**索引表的访问**，预排序使用了一个行号到叶子节点号的索引表（row_idx_to_tree_node_idx），来*防止数据切分时对所有的数据进行切分*，即**只对该叶子节点上的样本切分**。在与level-wise进行结合的时候， 每一个叶子节点都要切分数据，这也是随机的访问。这样会带来严重的系统性能下降。

而直方图算法则是天然的cache friendly。在直方图算法的第3个for循环的时候，就已经统计好了每个bin的梯度，因此，在计算gain的时候，只需要对bin进行访问，造成的cache-miss问题会小很多。

最后，在数据并行的时候，用 histgoram 可以**大幅降低通信代价**。用 pre-sorted 算法的话，通信代价是非常大的(几乎是没办法用的)。所以 xgboost在并行的时候也使用 histogram进行通信。


**3.2 劣势**

histogram算法不能找到很精确的分割点，训练误差没有pre-sorted好。但从实验结果来看， histogram算法在测试集的误差和pre-sorted算法**差异并不是很大**，甚至有时候效果更好。实际上可能决策树对于分割点的精确程度并不太敏感，而且较“粗”的分割点也自带正则化的效果，再加上boosting算法本身就是弱分类器的集成。

**3.3 Histogram直方算法**

直方图算法的基本思想是先把**连续的浮点特征值离散化成k个整数**，同时构造一个宽度为k的直方图。在遍历数据的时候，根据离散化后的值作为索引在直方图中累积统计量，当遍历一次数据后，直方图累积了需要的统计量，然后根据直方图的离散值，遍历寻找最优的分割点。
![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/lgb-histogram-bin1.png)
 
使用直方图算法有很多优点。首先，最明显就是**内存消耗的降低**，直方图算法不仅**不需要额外存储预排序的结果**，而且可以**只保存特征离散化后的值**，而这个值一般用8位整型存储就足够了，内存消耗可以降低为原来的1/8。
![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/lgb-histogram-bin2.png)
 
然后在计算上的代价也大幅降低，**预排序算法每遍历一个特征值就需要计算一次分裂的增益**，而直方图算法只需要计算k次（k可以认为是常数），时间复杂度从O(#data*#feature)优化到O(k*#features)。 　

当然，Histogram 算法并不是完美的。由于特征被离散化后，找到的**并不是很精确的分割点**，所以会对结果产生影响。但在不同的数据集上的结果表明，离散化的分割点对最终的精度影响并不是很大，甚至有时候会更好一点。
>原因是决策树本来就是**弱模型，分割点是不是精确并不是太重要**；**较粗的分割点也有正则化的效果，可以有效地防止过拟合**；即使单棵树的训练误差比精确分割的算法稍大，但在梯度提升（Gradient Boosting）的框架下没有太大的影响。

**3.4 直方图加速**
- LightGBM 另一个优化是 Histogram（直方图）做差加速。一个容易观察到的现象：**一个叶子的直方图可以由它的父亲节点的直方图与它兄弟的直方图做差得到**。通常构造直方图，需要遍历该叶子上的所有数据，但直方图做差仅需遍历直方图的k个桶。利用这个方法，LightGBM 可以在构造一个叶子的直方图后，可以用非常微小的代价得到它兄弟叶子的直方图，在速度上可以提升一倍。
 ![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/lgb-histogram-accelerate.png)

## 4.leaf-wise VS level-wise

在 Histogram 算法之上，LightGBM 进行进一步的优化。首先它抛弃了大多数 GBDT 工具使用的按层生长 (level-wise) 的决策树生长策略，而使用了带有深度限制的按叶子生长 (leaf-wise) 算法。

Level-wise 过一次数据可以**同时分裂同一层的叶子，容易进行多线程优化，也好控制模型复杂度，不容易过拟合**。但实际上 Level-wise 是一种低效的算法，因为它**不加区分的对待同一层的叶子，带来了很多没必要的开销**，因为实际上**很多叶子的分裂增益较低**，没必要进行搜索和分裂。
 ![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/lgb-level-wise-tree.png)


Leaf-wise 则是一种更为高效的策略，**每次从当前所有叶子中，找到分裂增益最大的一个叶子，然后分裂，如此循环**。因此同Level-wise相比，在分裂次数相同的情况下，Leaf-wise 可以降低更多的误差，得到更好的精度。Leaf-wise 的缺点是可能会长出比较深的决策树，产生过拟合。因此 LightGBM 在 Leaf-wise 之上增加了一个最大深度的限制，在保证高效率的同时防止过拟合。
 ![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/lgb-leaf-wise-tree.png)

## 5.特征并行和数据并行

LightGBM 原生支持并行学习，目前支持**特征并行和数据并行的两种**：

- 特征并行的主要思想是在**不同机器在不同的特征集合上分别寻找最优的分割点**，然后在**机器间同步最优的分割点**。
- 数据并行则是让**不同的机器先在本地构造直方图，然后进行全局的合并**，最后在**合并的直方图上面寻找最优分割点**。

LightGBM 针对这两种并行方法都做了优化：

1. 在特征并行算法中，通过在本地保存全部数据**避免对数据切分结果的通信**；

2. 在数据并行中使用分散规约 (Reduce scatter) **把直方图合并的任务分摊到不同的机器，降低通信和计算，并利用直方图做差，进一步减少了一半的通信量**。基于投票的数据并行则进一步优化数据并行中的通信代价，使通信代价变成常数级别。在数据量很大的时候，使用投票并行可以得到非常好的加速效果。
 ![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/lgb-feature-parallel-opsimiter1.jpg)
 ![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/lgb-feature-parallel-optimiter2.jpg)
 ![formula](https://raw.githubusercontent.com/xmj-datawhale/adv-algorithm/master/img/lgb-feature-parallel-opsimiter3.jpg)

**注意**
- 当生长相同的叶子时，Leaf-wise 比 level-wise 减少更多的损失。
- 高速，高效处理大数据，运行时需要更低的内存，支持 GPU
- 不要在少量数据上使用，会过拟合，建议 10,000+ 行记录时使用。

------------
特征并行
传统算法

传统的特征并行算法旨在于在并行化决策树中的“ Find Best Split.主要流程如下:

垂直划分数据（不同的机器有不同的特征集）
在本地特征集寻找最佳划分点 {特征, 阈值}
本地进行各个划分的通信整合并得到最佳划分
以最佳划分方法对数据进行划分，并将数据划分结果传递给其他线程
其他线程对接受到的数据进一步划分

传统的特征并行方法主要不足:

存在计算上的局限，传统特征并行无法加速 “split”（时间复杂度为 “O（#data）”）。 因此，当数据量很大的时候，难以加速。
需要对划分的结果进行通信整合，其额外的时间复杂度约为 “O（#data/8）”（一个数据一个字节）

LightGBM 中的特征并行

既然在数据量很大时，传统数据并行方法无法有效地加速，我们做了一些改变：不再垂直划分数据，即每个线程都持有全部数据。 因此，LighetGBM中没有数据划分结果之间通信的开销，各个线程都知道如何划分数据。 而且，“#data” 不会变得更大，所以，在使每天机器都持有全部数据是合理的。

LightGBM 中特征并行的流程如下：

每个线程都在本地数据集上寻找最佳划分点｛特征， 阈值｝
本地进行各个划分的通信整合并得到最佳划分
执行最佳划分

然而，该特征并行算法在数据量很大时仍然存在计算上的局限。因此，建议在数据量很大时使用数据并行。
数据并行
传统算法

数据并行旨在于并行化整个决策学习过程。数据并行的主要流程如下：

水平划分数据
线程以本地数据构建本地直方图
将本地直方图整合成全局整合图
在全局直方图中寻找最佳划分，然后执行此划分

传统数据划分的不足：

高通讯开销。 如果使用点对点的通讯算法，一个机器的通讯开销大约为 “O(#machine * #feature * #bin)” 。 如果使用集成的通讯算法（例如， “All Reduce”等），通讯开销大约为 “O(2 * #feature * #bin)”。

LightGBM中的数据并行

LightGBM 中采用以下方法较少数据并行中的通讯开销：

不同于“整合所有本地直方图以形成全局直方图”的方式，LightGBM 使用分散规约(Reduce scatter)的方式对不同线程的不同特征（不重叠的）进行整合。 然后线程从本地整合直方图中寻找最佳划分并同步到全局的最佳划分中。
如上所述。LightGBM 通过直方图做差法加速训练。 基于此，我们可以进行单叶子的直方图通讯，并且在相邻直方图上使用做差法。

通过上述方法，LightGBM 将数据并行中的通讯开销减少到 “O(0.5 * #feature * #bin)”。




## 6.顺序访问梯度
预排序算法中有两个频繁的操作会导致cache-miss，也就是缓存消失(对速度的影响很大，特别是数据量很大的时候，顺序访问比随机访问的速度快4倍以上)。

1. 对梯度的访问：在计算增益的时候需要利用梯度，对于不同的特征，访问梯度的顺序是不一样的，并且是随机的。
2.  对于索引表的访问：预排序算法使用了行号和叶子节点号的索引表，防止数据切分的时候对所有的特征进行切分。同访问梯度一样，所有的特征都要通过访问这个索引表来索引。

这两个操作都是随机的访问，会给系统性能带来非常大的下降。

LightGBM使用的直方图算法能很好的解决这类问题。首先。对梯度的访问，因为不用对特征进行排序，同时，所有的特征都用同样的方式来访问，所以只需要对梯度访问的顺序进行重新排序，所有的特征都能连续的访问梯度。并且直方图算法不需要把数据id到叶子节点号上（不需要这个索引表，没有这个缓存消失问题）

## 7.支持类别特征

实际上大多数机器学习工具都无法直接支持类别特征，一般需要把类别特征，转化到多维的0/1 特征，降低了空间和时间的效率。而类别特征的使用是在实践中很常用的。基于这个考虑，**LightGBM 优化了对类别特征的支持，可以直接输入类别特征，不需要额外的0/1 展开**。并在决策树算法上增加了类别特征的决策规则。在 Expo 数据集上的实验，相比0/1展开的方法，训练速度可以加速 8 倍，并且精度一致。据我们所知，LightGBM 是第一个直接支持类别特征的 GBDT工具。

LightGBM 的单机版本还有很多其他细节上的优化，比如 cache 访问优化，多线程优化，稀疏特征优化等等。优化汇总如下：
 
<table>
<thead>
<tr>
  <th align="center">类目</th>
  <th align="center">预排序算法预排序算法</th>
  <th align="center">lightGBM</th>
</tr>
</thead>
<tbody><tr>
  <td align="center">内存占用</td>
  <td align="center"><code>2*#feature*#data*4Bytes</code></td>
  <td align="center"><code>*#feature*#data*1Bytes</code></td>
</tr>
<tr>
  <td align="center">统计量累积</td>
  <td align="center"><code>O(*#feature*#data)</code></td>
  <td align="center"><code>O(*#feature*#data)</code></td>
</tr>
<tr>
  <td align="center">分割增益计算</td>
  <td align="center"><code>O(*#feature*#data)</code></td>
  <td align="center"><code>O(*#feature*#k)</code></td>
</tr>
<tr>
  <td align="center">直方图做差</td>
  <td align="center">N/A</td>
  <td align="center">加速一倍</td>
</tr>
<tr>
  <td align="center">直接支持类别特征</td>
  <td align="center">N/A</td>
  <td align="center">在Expo数据上加速8倍</td>
</tr>
<tr>
  <td align="center">Cache优化</td>
  <td align="center">N/A</td>
  <td align="center">在Higgs数据上加速40%</td>
</tr>
<tr>
  <td align="center">带深度限制的Leaf-wise的决策树算法</td>
  <td align="center">N/A</td>
  <td align="center">精度更</td>
</tr>
</tbody></table>

## 8.应用场景
支持以下应用:
 回归，目标函数为 L2 loss
 二分类， 目标函数为 logloss（对数损失）
 多分类
 lambdarank, 目标函数为基于 NDCG 的 lambdarank

## 9. sklearn参数

In [0]:
class lightgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1,
                              learning_rate=0.1, n_estimators=100, subsample_for_bin=200000,
                              objective=None, class_weight=None, min_split_gain=0.0, 
                              min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
                              subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, 
                              reg_lambda=0.0, random_state=None, n_jobs=-1, silent=True, 
                              importance_type='split', **kwargs)


**参数**

`boosting_type`：优化方法

`num_leaves`：决策树最大叶结点数量

`max_depth`：决策树最大深度

`learning_rate`：学习率

`n_estimators`：基础学习器数量

`subsample_for_bin`：确定binning边界时时考虑的样本数量

`objective`：优化函数

`class_weight`：类别权重

`min_split_gain`：节点继续分裂最小损失函数下降

`min_child_weight`：叶结点最小样本权重

`min_child_samples`：叶结点最小样本数量

`subsample`：样本抽样

`subsample_freq`：样本抽样

`colsample_bytree`：特征抽样

`reg_alpha`：正则化变量

`reg_lambda`：正则化变量

`random_state`：随机种子

`n_jobs`：控制并行

`silent`：控制输出

`importance_type`：特征重要程度计算方式

### 3.1 控制参数
<table>
<thead>
<tr>
  <th>Control Parameters</th>
  <th>含义</th>
  <th>用法</th>
</tr>
</thead>
<tbody><tr>
  <td><code>max_depth</code></td>
  <td>树的最大深度</td>
  <td>当模型过拟合时,可以考虑首先降低 <code>max_depth</code></td>
</tr>
<tr>
  <td><code>min_data_in_leaf</code></td>
  <td>叶子可能具有的最小记录数</td>
  <td>默认20，过拟合时用</td>
</tr>
<tr>
  <td><code>feature_fraction</code></td>
  <td>例如 为0.8时，意味着在每次迭代中随机选择80％的参数来建树</td>
  <td>boosting 为 random forest 时用</td>
</tr>
<tr>
  <td><code>bagging_fraction</code></td>
  <td>每次迭代时用的数据比例</td>
  <td>用于加快训练速度和减小过拟合</td>
</tr>
<tr>
  <td><code>early_stopping_round</code></td>
  <td>如果一次验证数据的一个度量在最近的<code>early_stopping_round</code> 回合中没有提高，模型将停止训练</td>
  <td>加速分析，减少过多迭代</td>
</tr>
<tr>
  <td>lambda</td>
  <td>指定正则化</td>
  <td>0～1</td>
</tr>
<tr>
  <td><code>min_gain_to_split</code></td>
  <td>描述分裂的最小 gain</td>
  <td>控制树的有用的分裂</td>
</tr>
<tr>
  <td><code>max_cat_group</code></td>
  <td>在 group 边界上找到分割点</td>
  <td>当类别数量很多时，找分割点很容易过拟合时</td>
</tr>
</tbody></table>

### 3.2 核心参数
<table>
<thead>
<tr>
  <th>CoreParameters</th>
  <th>含义</th>
  <th>用法</th>
</tr>
</thead>
<tbody><tr>
  <td>Task</td>
  <td>数据的用途</td>
  <td>选择 train 或者 predict</td>
</tr>
<tr>
  <td>application</td>
  <td>模型的用途</td>
  <td>选择 regression: 回归时，binary: 二分类时，multiclass: 多分类时</td>
</tr>
<tr>
  <td>boosting</td>
  <td>要用的算法</td>
  <td>gbdt， rf: random forest， dart: Dropouts meet Multiple Additive Regression Trees， goss: <code>Gradient-based One-Side Sampling</code></td>
</tr>
<tr>
  <td><code>num_boost_round</code></td>
  <td>迭代次数</td>
  <td>通常 100+</td>
</tr>
<tr>
  <td><code>learning_rate</code></td>
  <td>如果一次验证数据的一个度量在最近的 <code>early_stopping_round</code> 回合中没有提高，模型将停止训练</td>
  <td>常用 0.1, 0.001, 0.003…</td>
</tr>
<tr>
  <td><code>num_leaves</code></td>
  <td></td>
  <td>默认 31</td>
</tr>
<tr>
  <td>device</td>
  <td></td>
  <td>cpu 或者 gpu</td>
</tr>
<tr>
  <td>metric</td>
  <td></td>
  <td><code>mae: mean absolute error ， mse: mean squared error ， binary_logloss: loss for binary classification ， multi_logloss: loss for multi classification</code></td>
</tr>
</tbody></table>

### 3.3 IO参数
<table>
<thead>
<tr>
  <th>IO parameter</th>
  <th>含义</th>
</tr>
</thead>
<tbody><tr>
  <td><code>max_bin</code></td>
  <td>表示 feature 将存入的 bin 的最大数量</td>
</tr>
<tr>
  <td><code>categorical_feature</code></td>
  <td>如果 <code>categorical_features = 0,1,2</code>， 则列 0，1，2是 categorical 变量</td>
</tr>
<tr>
  <td><code>ignore_column</code></td>
  <td>与 <code>categorical_features</code> 类似，只不过不是将特定的列视为categorical，而是完全忽略</td>
</tr>
<tr>
  <td><code>save_binary</code></td>
  <td>这个参数为 true 时，则数据集被保存为二进制文件，下次读数据时速度会变快</td>
</tr>
</tbody></table>

### 3.4 调参
<table>
<thead>
<tr>
  <th>IO parameter</th>
  <th>含义</th>
</tr>
</thead>
<tbody><tr>
  <td><code>num_leaves</code></td>
  <td>取值应 <code>&lt;= 2 ^（max_depth）</code>， 超过此值会导致过拟合</td>
</tr>
<tr>
  <td><code>min_data_in_leaf</code></td>
  <td>将它设置为较大的值可以避免生长太深的树，但可能会导致 underfitting，在大型数据集时就设置为数百或数千</td>
</tr>
<tr>
  <td><code>max_depth</code></td>
  <td>这个也是可以限制树的深度</td>
</tr>
</tbody></table>

下表对应了 Faster Speed ，better accuracy ，over-fitting 三种目的时，可以调的参数

<table>
<thead>
<tr>
  <th>Faster Speed</th>
  <th>better accuracy</th>
  <th>over-fitting</th>
</tr>
</thead>
<tbody><tr>
  <td>将 <code>max_bin</code> 设置小一些</td>
  <td>用较大的 <code>max_bin</code></td>
  <td><code>max_bin</code> 小一些</td>
</tr>
<tr>
  <td></td>
  <td><code>num_leaves</code> 大一些</td>
  <td><code>num_leaves</code> 小一些</td>
</tr>
<tr>
  <td>用 <code>feature_fraction</code> 来做 <code>sub-sampling</code></td>
  <td></td>
  <td>用 <code>feature_fraction</code></td>
</tr>
<tr>
  <td>用 <code>bagging_fraction 和 bagging_freq</code></td>
  <td></td>
  <td>设定 <code>bagging_fraction 和 bagging_freq</code></td>
</tr>
<tr>
  <td></td>
  <td>training data 多一些</td>
  <td>training data 多一些</td>
</tr>
<tr>
  <td>用 <code>save_binary</code> 来加速数据加载</td>
  <td>直接用 categorical feature</td>
  <td>用 <code>gmin_data_in_leaf 和 min_sum_hessian_in_leaf</code></td>
</tr>
<tr>
  <td>用 parallel learning</td>
  <td>用 dart</td>
  <td>用 <code>lambda_l1, lambda_l2 ，min_gain_to_split</code> 做正则化</td>
</tr>
<tr>
  <td></td>
  <td><code>num_iterations</code> 大一些，<code>learning_rate</code> 小一些</td>
  <td>用 <code>max_depth</code> 控制树的深度</td>
</tr>
</tbody></table>

## 10.CatBoost(了解)
https://zhuanlan.zhihu.com/p/37916954